# Unsupervised Anomaly Detection Brain-MRI

Jupyter notebook for running all the experiments from our [paper](https://arxiv.org/abs/2004.03271). 

Hyperparameters may have to be adjusted!

## Preperation

Installing tensorflow version 1 manually, since it is not supported anymore in colab. 


In [1]:
# !pip uninstall -y tensorflow 
# !pip install tensorflow-gpu==1.15
# !apt install --allow-change-held-packages libcudnn7=7.6.5.32-1+cuda10.2

### Get Code
Clone Code from github.com

In [2]:
# ! git clone https://github.com/StefanDenn3r/Unsupervised_Anomaly_Detection_Brain_MRI

In [3]:
# cd Unsupervised_Anomaly_Detection_Brain_MRI/

Installing all the requirements. 

Note: Although Colab might say, that a restart of the runtime is necessary, for us it was not. 

In [4]:
# ! pip install -r requirements.txt

## Download Brainweb Dataset

Downloading the brainweb dataset from [here](https://brainweb.bic.mni.mcgill.ca/). 
The authors request optionally to provide your name, institution and email. 

In [5]:
from utils.brainweb_download import download_brainweb_dataset
from pathlib import Path

download_brainweb_dataset(
    base_dir=Path('./content/data/Brainweb'),
    name="",
    institution="",
    email=""
)

Skipping content/data/Brainweb/lesions/severe/t2_ai_msles2_1mm_pn0_rf0.mnc.gz download, since it already exists.
Skipping conversion of content/data/Brainweb/lesions/severe/t2_ai_msles2_1mm_pn0_rf0.mnc.gz, since content/data/Brainweb/lesions/severe/t2_ai_msles2_1mm_pn0_rf0.nii.gz already exists.
Skipping content/data/Brainweb/lesions/severe/t2_ai_msles2_1mm_pn0_rf20.mnc.gz download, since it already exists.
Skipping conversion of content/data/Brainweb/lesions/severe/t2_ai_msles2_1mm_pn0_rf20.mnc.gz, since content/data/Brainweb/lesions/severe/t2_ai_msles2_1mm_pn0_rf20.nii.gz already exists.
Skipping content/data/Brainweb/lesions/severe/t2_ai_msles2_1mm_pn0_rf40.mnc.gz download, since it already exists.
Skipping conversion of content/data/Brainweb/lesions/severe/t2_ai_msles2_1mm_pn0_rf40.mnc.gz, since content/data/Brainweb/lesions/severe/t2_ai_msles2_1mm_pn0_rf40.nii.gz already exists.
Skipping content/data/Brainweb/lesions/severe/t2_ai_msles2_1mm_pn1_rf0.mnc.gz download, since it alread

## Training

### Imports

In [6]:
import json
import os
import tensorflow as tf
from datetime import datetime
from utils.default_config_setup import get_config, get_options, get_datasets
from trainers.AE import AE
from trainers.VAE import VAE
from trainers.CE import CE
from trainers.ceVAE import ceVAE
from trainers.VAE_You import VAE_You
from trainers.GMVAE import GMVAE
from trainers.GMVAE_spatial import GMVAE_spatial

# from trainers.fAnoGAN import fAnoGAN
from trainers.ConstrainedAAE import ConstrainedAAE
from trainers.ConstrainedAE import ConstrainedAE

# from trainers.AnoVAEGAN import AnoVAEGAN
# from models import (
#     autoencoder,
#     variational_autoencoder,
#     context_encoder_variational_autoencoder,
#     variational_autoencoder_Zimmerer,
#     context_encoder_variational_autoencoder,
#     context_encoder_variational_autoencoder_Zimmerer,
#     gaussian_mixture_variational_autoencoder_You,
#     gaussian_mixture_variational_autoencoder_spatial,
#     gaussian_mixture_variational_autoencoder,
#     fanogan,
#     fanogan_schlegl,
#     constrained_autoencoder,
#     constrained_adversarial_autoencoder,
#     constrained_adversarial_autoencoder_Chen,
#     anovaegan,
# )
from models import (
    autoencoder,
    variational_autoencoder,
    context_encoder_variational_autoencoder,
    variational_autoencoder_Zimmerer,
    context_encoder_variational_autoencoder,
    context_encoder_variational_autoencoder_Zimmerer,
    gaussian_mixture_variational_autoencoder_You,
    gaussian_mixture_variational_autoencoder_spatial,
    gaussian_mixture_variational_autoencoder,
    fanogan,
    # fanogan_schlegl,
    constrained_autoencoder,
    constrained_adversarial_autoencoder,
    # constrained_adversarial_autoencoder_Chen,
    anovaegan,
)
from utils import Evaluation
from utils.default_config_setup import get_config, get_options, get_datasets, Dataset

Set paths to datasets and where to save checkpoints and evaluations.

**Note:** You may have to adjust `dataset_root` and `save_dir`

In [7]:
def get_CONFIG(timestamp=None):
  current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
  if timestamp:
    current_time=timestamp
  dataset_root = "./content/data"
  save_dir = "./content/saved"
  CONFIG = {
    "BRAINWEBDIR": os.path.join(dataset_root, 'Brainweb'),
#    "MSSEG2008DIR": os.path.join(dataset_root, 'MSSEG2008'),
#    "MSISBI2015DIR": os.path.join(dataset_root, 'ISBIMSlesionChallenge'),
#    "MSLUBDIR": os.path.join(dataset_root, 'MSlub'),
    "CHECKPOINTDIR": os.path.join(save_dir, 'checkpoints', current_time),
    "SAMPLEDIR": os.path.join(save_dir, 'sample_dir', current_time),
  }
  return CONFIG

### Manual Training


#### Baseline

In [10]:
# tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=AE, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)
print(config)

# Create an instance of the model and train it
model = AE(tf.compat.v1.Session(), config, network=autoencoder.autoencoder)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))


#Params in Bottleneck: 267536
#Params in Encoder: 667456
#Params in Decoder: 692257
#Params in total: 1627249


2024-11-13 08:49:56.931536: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch (TRAIN): [ 0] [   0/   7] loss: 2838.44775391
Epoch (TRAIN): [ 0] [   1/   7] loss: 2978.34716797
Epoch (TRAIN): [ 0] [   2/   7] loss: 3029.44433594
Epoch (TRAIN): [ 0] [   3/   7] loss: 2845.94531250
Epoch (TRAIN): [ 0] [   4/   7] loss: 3053.91186523
Epoch (TRAIN): [ 0] [   5/   7] loss: 2872.05908203
Epoch (TRAIN): [ 0] [   6/   7] loss: 2737.60375977
Epoch (VAL): [ 0] [   0/   2] loss: 2555.56567383
Epoch (VAL): [ 0] [   1/   2] loss: 2694.03613281
Epoch (TRAIN): [ 1] [   0/   7] loss: 3053.90551758
Epoch (TRAIN): [ 1] [   1/   7] loss: 2925.19531250
Epoch (TRAIN): [ 1] [   2/   7] loss: 2900.31835938
Epoch (TRAIN): [ 1] [   3/   7] loss: 2908.61840820
Epoch (TRAIN): [ 1] [   4/   7] loss: 2819.11718750
Epoch (TRAIN): [ 1] [   5/   7] loss: 2773.35302734
Epoch (TRAIN): [ 1] [   6/   7] loss: 2548.50097656
Epoch (VAL): [ 1] [   0/   2] loss: 1970.12231445
Epoch (VAL): [ 1] [   1/   2] loss: 1943.

KeyboardInterrupt: 

**VAE**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=VAE, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

# Create an instance of the model and train it
model = VAE(tf.compat.v1.Session(), config, network=variational_autoencoder.variational_autoencoder)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))


#### ceVAE - Variations

Paper: [Context-encoding Variational Autoencoder for Unsupervised Anomaly Detection](https://arxiv.org/abs/1812.05941)



**CE**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=CE, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

# Create an instance of the model and train it
model = CE(tf.compat.v1.Session(), config, network=autoencoder.autoencoder)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))


**ceVAE**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=ceVAE, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

config.use_gradient_based_restoration = 0.002

# Create an instance of the model and train it
model = ceVAE(tf.compat.v1.Session(), config, network=context_encoder_variational_autoencoder.context_encoder_variational_autoencoder)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))


**VAE-Zimmerer**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=64, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=VAE, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

# Create an instance of the model and train it
model = VAE(tf.compat.v1.Session(), config, network=variational_autoencoder_Zimmerer.variational_autoencoder_Zimmerer)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))


**ceVAE-Zimmerer**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=64, learningrate=0.0001, numEpochs=1, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=ceVAE, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

# Create an instance of the model and train it
model = ceVAE(tf.compat.v1.Session(), config, network=context_encoder_variational_autoencoder_Zimmerer.context_encoder_variational_autoencoder_Zimmerer)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))


#### GMVAE-(Restoration)-Variations

Paper: [Unsupervised Lesion Detection via Image Restoration with a Normative Prior](https://openreview.net/forum?id=S1xg4W-leV)


**VAE-You**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=VAE_You, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

config.restore_lr = 1e-3
config.restore_steps = 10
config.tv_lambda = 0.0

# Create an instance of the model and train it
model = VAE_You(tf.compat.v1.Session(), config, network=variational_autoencoder.variational_autoencoder)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))


**GMVAE-You**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=GMVAE_spatial, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

config.dim_c = 9
config.dim_z = 1
config.dim_w = 1
config.c_lambda = 1
config.restore_lr = 1e-3
config.restore_steps = 10
config.tv_lambda = 1

# Create an instance of the model and train it
model = GMVAE_spatial(tf.compat.v1.Session(), config, network=gaussian_mixture_variational_autoencoder_You.gaussian_mixture_variational_autoencoder_You)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))


**GMVAE**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=GMVAE, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

config.dim_c = 9
config.dim_z = 128
config.dim_w = 1
config.c_lambda = 1
config.restore_lr = 1e-3
config.restore_steps = 10
config.tv_lambda = 0.0

# Create an instance of the model and train it
model = GMVAE(tf.compat.v1.Session(), config, network=gaussian_mixture_variational_autoencoder.gaussian_mixture_variational_autoencoder)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))

**GMVAE-spatial**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=GMVAE_spatial, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

config.dim_c = 9
config.dim_z = 1
config.dim_w = 1
config.c_lambda = 1
config.restore_lr = 1e-3
config.restore_steps = 10
config.tv_lambda = 0.0

# Create an instance of the model and train it
model = GMVAE_spatial(tf.compat.v1.Session(), config, network=gaussian_mixture_variational_autoencoder_spatial.gaussian_mixture_variational_autoencoder_spatial)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))

#### f-AnoGAN

Paper: [f-AnoGAN: Fast unsupervised anomaly detection with generative adversarial networks.](https://www.ncbi.nlm.nih.gov/pubmed/30831356)

**Unified f-AnoGan**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=fAnoGAN, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

config.kappa = 1.0
config.scale = 10.0

# Create an instance of the model and train it
model = fAnoGAN(tf.Session(), config, network=fanogan.fanogan)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))

**f-AnoGAN - Schlegl**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=8, learningrate=0.0001, numEpochs=2, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=fAnoGAN, options=options, optimizer='ADAM', intermediateResolutions=[16, 16], dropout_rate=0.1, dataset=datasetHC)

config.kappa = 1.0
config.scale = 10.0

# Create an instance of the model and train it
model = fAnoGAN(tf.Session(), config, network=fanogan_schlegl.fanogan_schlegl)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))


#### Constrained Adversarial AE

Paper: [Unsupervised Detection of Lesions in Brain MRI using constrained adversarial auto-encoders](https://arxiv.org/abs/1806.04972)

**constrained AAE**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=ConstrainedAAE, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

config.scale = 10.0
config.rho = 1.0

# Create an instance of the model and train it
model = ConstrainedAAE(tf.Session(), config, network=constrained_adversarial_autoencoder.constrained_adversarial_autoencoder)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))


**constrained AAE Chen**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=8, learningrate=0.0001, numEpochs=2, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=ConstrainedAAE, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

config.kappa = 1.0
config.scale = 10.0
config.rho = 1.0

# Create an instance of the model and train it
model = ConstrainedAAE(tf.Session(), config, network=constrained_adversarial_autoencoder_Chen.constrained_adversarial_autoencoder_Chen)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))

#### AnoVAEGAN

Paper: [Deep autoencoding models for unsupervised anomaly segmentation in brain MR images](https://arxiv.org/abs/1804.04488)

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=AnoVAEGAN, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

# Create an instance of the model and train it
model = AnoVAEGAN(tf.Session(), config, network=anovaegan.anovaegan)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))
